In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [2]:
#get the data
for dirpath, dirnames, filenames in os.walk('../')

'wget' is not recognized as an internal or external command,
operable program or batch file.
